In [28]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup
import re
!pip install fuzzywuzzy
!pip install sentence_transformers
from fuzzywuzzy import fuzz
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import sparse
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, log_loss, accuracy_score, precision_score, recall_score, roc_auc_score, f1_score
from sklearn.calibration import CalibratedClassifierCV
import xgboost as xgb
import nltk
import time
from matplotlib.pyplot import figure
from sentence_transformers import SentenceTransformer
from sklearn.preprocessing import MinMaxScaler
from joblib import dump, load
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.model_selection import ShuffleSplit
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
# import optuna
import hyperopt
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
import warnings
import gc
from sklearn.model_selection import cross_val_score
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
from google.colab import drive
drive.mount('/content/drive')

!ls "/content/drive/My Drive/Quora-Questions/"

Mounted at /content/drive
 new_boolq.jsonl   quora_train.csv  'test (1).csv'   train.csv.zip
 quora_test.csv    results	     test.csv


In [54]:
# data = pd.read_csv('/content/drive/My Drive/Quora-Questions/train.csv.zip')
train = pd.read_csv('/content/drive/My Drive/Quora-Questions/quora_train.csv')
test = pd.read_csv('/content/drive/My Drive/Quora-Questions/quora_test.csv')
y_train = train.is_duplicate
y_test = test.is_duplicate
train = train.drop(columns=['is_duplicate'])
test = test.drop(columns=['is_duplicate'])
print(train.shape, test.shape)

(283003, 10) (121287, 10)


In [4]:
# Get cosine similarity and euclidean distance between two vectors
def cosine_euclidean(u, v):
    return np.array([np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v)), np.linalg.norm(u - v)])

In [35]:
# open .npy files and loop through the sentence embeddings
with open('/content/drive/My Drive/Quora-Questions/results/temp_train_question1_sentenceBERT.npy', 'rb') as q1_vec, open('/content/drive/My Drive/Quora-Questions/results/temp_train_question2_sentenceBERT.npy', 'rb') as q2_vec:
    distances = []
    while True:
        try:
            q1_20k = np.load(q1_vec, allow_pickle=True)
            q2_20k = np.load(q2_vec, allow_pickle=True)
            for q1,q2 in zip(q1_20k, q2_20k):
                dists = cosine_euclidean(q1, q2)
                distances.append(dists)
        except IOError as e:
            distances = np.array(distances)
            break

In [36]:
distances = pd.DataFrame(distances, columns=['cosine_simlarity_bert', 'euclidean_distance_bert'])
train = pd.concat([train, pd.DataFrame(distances)], axis=1)
train

,question1_final,question2_final,q1_word_num,q2_word_num,total_word_num,differ_word_num,same_first_word,same_last_word,common_word_num,fuzz_ratio,cosine_simlarity_bert,euclidean_distance_bert
0,why has nokia failed in the market,why did nokia fail,7,4,11,3,1,0,2,58,0.796555,2.103234
1,what will life be like in 2100,what will the world look like in 2100,7,8,15,1,1,1,5,75,0.909718,1.470620
2,has disney tv gone good,could disney make a good dr seuss movie,5,8,13,3,0,0,2,45,0.486789,3.408969
3,what are the reasons behind the poor performan...,why india is big loser in olympics,12,7,19,5,0,1,3,39,0.852298,1.774711
4,which is the best phone i can buy in the range...,mobile phones what is the best phone i can bu...,15,17,32,2,0,0,10,63,0.869250,1.685515
...,...,...,...,...,...,...,...,...,...,...,...,...
282998,which is better hp pavilion 15 ab552tx or au0...,which is better hp pavilion 15 ab522tx or hp p...,9,12,21,3,1,0,7,77,0.932627,1.200367
282999,what are some of the best cars i can get for ...,what is the best car under dollar 20k,13,8,21,5,1,0,4,70,0.630013,2.911153
283000,what is the most valuable thing you should do ...,what is the most valuable thing for you,12,8,20,4,1,0,7,73,0.658210,2.816115
283001,how do i prepare for gre in 45 days and score 315,what is the best way to prepare for the gre in...,12,17,29,5,0,1,8,67,0.948736,1.080289


In [37]:
# open .npy files and loop through the sentence embeddings
with open('/content/drive/My Drive/Quora-Questions/results/temp_test_question1_sentenceBERT.npy', 'rb') as q1_vec, open('/content/drive/My Drive/Quora-Questions/results/temp_test_question2_sentenceBERT.npy', 'rb') as q2_vec:
    distances = []
    while True:
        try:
            q1_20k = np.load(q1_vec, allow_pickle=True)
            q2_20k = np.load(q2_vec, allow_pickle=True)
            for q1,q2 in zip(q1_20k, q2_20k):
                dists = cosine_euclidean(q1, q2)
                distances.append(dists)
        except IOError as e:
            distances = np.array(distances)
            break
distances = pd.DataFrame(distances, columns=['cosine_simlarity_bert', 'euclidean_distance_bert'])
test = pd.concat([test, pd.DataFrame(distances)], axis=1)

In [38]:
train.drop(columns=['question1_final', 'question2_final'], inplace=True)
train = train.to_numpy()
scaler = MinMaxScaler()
scaler.fit(train)
train = scaler.transform(train)

In [39]:
test.drop(columns=['question1_final', 'question2_final'], inplace=True)
testdata = test.to_numpy()
testdata = scaler.transform(testdata)

In [40]:
with open('temp_testdata.npy', 'wb') as f:
    batch = 20000
    while(len(testdata)):
        tempdata = testdata[:batch]
        testdata = testdata[batch:]
        np.save(f, tempdata, allow_pickle=True)

In [11]:
!cp temp_testdata.npy /content/drive/MyDrive/Quora-Questions/results

In [41]:
def loadVectors(filename):
    with open(filename, 'rb') as f:
        q_vectors = []
        while True:
            try:
                q_vec = np.load(f, allow_pickle=True)
                q_vectors.extend(list(q_vec))
            except IOError as e:
                q_vectors = np.array(q_vectors)
                break
    return q_vectors

In [42]:
train_question1_vec = loadVectors('/content/drive/My Drive/Quora-Questions/results/temp_train_question1_sentenceBERT.npy')
train_question2_vec = loadVectors('/content/drive/My Drive/Quora-Questions/results/temp_train_question2_sentenceBERT.npy')

In [43]:
train = np.hstack((train_question1_vec, train_question2_vec))
train.shape

(283003, 1536)

In [44]:
splits = ShuffleSplit(n_splits=1, test_size=.3, random_state=42)
svc_param_grid = {'C':[0.01, 0.1, 1, 10, 100]}
svc_clf = LinearSVC(penalty='l2', loss='squared_hinge', dual=False, max_iter=3000)
svc_clf_search = HalvingGridSearchCV(svc_clf, svc_param_grid, cv=splits, factor=2, scoring='accuracy', verbose=3)

In [45]:
svc_clf_search.fit(train, y_train)

n_iterations: 3
n_required_iterations: 3
n_possible_iterations: 3
min_resources_: 70750
max_resources_: 283003
aggressive_elimination: False
factor: 2
----------
iter: 0
n_candidates: 5
n_resources: 70750
Fitting 1 folds for each of 5 candidates, totalling 5 fits
[CV 1/1] END ........C=0.01;, score=(train=0.738, test=0.728) total time=  38.2s
[CV 1/1] END .........C=0.1;, score=(train=0.744, test=0.728) total time= 1.5min
[CV 1/1] END ...........C=1;, score=(train=0.744, test=0.727) total time= 1.7min
[CV 1/1] END ..........C=10;, score=(train=0.743, test=0.728) total time= 2.4min
[CV 1/1] END .........C=100;, score=(train=0.743, test=0.728) total time= 2.3min
----------
iter: 1
n_candidates: 3
n_resources: 141500
Fitting 1 folds for each of 3 candidates, totalling 3 fits
[CV 1/1] END ..........C=10;, score=(train=0.737, test=0.730) total time= 4.0min
[CV 1/1] END .........C=100;, score=(train=0.737, test=0.730) total time= 3.8min
[CV 1/1] END ........C=0.01;, score=(train=0.734, test=

HalvingGridSearchCV(cv=ShuffleSplit(n_splits=1, random_state=42, test_size=0.3, train_size=None),
                    estimator=LinearSVC(dual=False, max_iter=3000), factor=2,
                    param_grid={'C': [0.01, 0.1, 1, 10, 100]},
                    scoring='accuracy', verbose=3)

In [46]:
svc_clf_model =  svc_clf_search.best_estimator_
svc_clf_model

LinearSVC(C=100, dual=False, max_iter=3000)

In [48]:
with open('/content/drive/My Drive/Quora-Questions/results/temp_testdata.npy', 'rb') as X_test_1, \
    open('/content/drive/My Drive/Quora-Questions/results/temp_test_question1_sentenceBERT.npy', 'rb') as X_test_q1, \
    open('/content/drive/My Drive/Quora-Questions/results/temp_test_question2_sentenceBERT.npy', 'rb') as X_test_q2:
    y_pred_proba_svc = []
    while True:
        try:
            test_20k = np.load(X_test_1, allow_pickle=True)
            q1_20k = np.load(X_test_q1, allow_pickle=True)
            q2_20k = np.load(X_test_q2, allow_pickle=True)
            X_test = np.hstack((q1_20k, q2_20k))
            y_pred_proba_svc.extend(list(svc_clf_model.predict(X_test)))
        except IOError as e:
            break


In [49]:
print("Scores for best SVC with settings: ", svc_clf_model)
print(accuracy_score(y_pred_proba_svc, y_test))
print(precision_score(y_pred_proba_svc, y_test))
print(recall_score(y_pred_proba_svc, y_test))
print(log_loss(y_pred_proba_svc, y_test))
print(roc_auc_score(y_pred_proba_svc, y_test))
print(f1_score(y_pred_proba_svc, y_test))

Scores for best SVC with settings:  LinearSVC(C=100, dual=False, max_iter=3000)
0.7322136750022673
0.5167792767514422
0.6776358391711309
9.249153919745595
0.7155430349750264
0.5863759662773964
